In [1]:
import pandas as pd

In [2]:
CAREER_LENGTH = 5

In [3]:
credible_authors = pd.read_csv('data/authors-scientific-atleast-'+ str(CAREER_LENGTH) + '-year_extended.csv', encoding='utf-8')

In [4]:
credible_authors.head()

,author,start_year,end_year,num_pub,career_length,early_career_prod,early_career_qual,early_career_rec,final_citation_count,gender
0,'maseka lesaoana,2001,2015,2,15,1,5.0,2,25.0,none
1,a aart blokhuis,1992,2005,2,14,1,0.0,0,0.0,none
2,a min tjoa,1979,2017,191,39,1,0.0,0,799.0,none
3,a-imam al-sammak,1992,2006,3,15,1,0.0,0,0.0,m
4,a-long jin,2011,2016,9,6,1,0.0,0,19.0,none
